<h1> Scrapping - Coleta de Dados

## Instalação e importação das bibliotecas necessárias

In [6]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
import os

### 📌 Etapa 1: Coleta de Conexões do LinkedIn

Nesta primeira etapa do processo, utilizamos perfis do LinkedIn previamente autenticados para extrair as conexões de cada usuário. Para automatizar essa tarefa, foi empregada a biblioteca **Selenium**, que permite a navegação automatizada em páginas web.

O processo executado compreende as seguintes ações:

1. **Acesso ao LinkedIn**  
   Inicia-se uma sessão no navegador e acessa-se a página inicial do LinkedIn.

2. **Autenticação**  
   Realiza-se o login automático utilizando as credenciais do usuário.

3. **Acesso à Página de Conexões**  
   Após o login, o script navega até a seção de conexões do perfil.

4. **Scroll Dinâmico da Página**  
   Como o LinkedIn carrega as conexões de forma incremental, o script executa *scrolls* sucessivos até o final da página, garantindo que todas as conexões sejam carregadas.

5. **Extração das URLs das Conexões**  
   São coletados os links (URLs) dos perfis de todas as conexões visíveis.

6. **Armazenamento dos Dados**  
   Todas as URLs extraídas são salvas em um arquivo `.txt` para uso posterior nas próximas etapas do processo.


In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.linkedin.com/login/pt")

EMAIL = "teste@gmail.com"
PASSWORD = "senha123"

driver.find_element(By.ID, "username").send_keys(EMAIL)
driver.find_element(By.ID, "password").send_keys(PASSWORD)
driver.find_element(By.XPATH, "//button[@type='submit']").click()
time.sleep(5)

driver.get("https://www.linkedin.com/mynetwork/invite-connect/connections/")
time.sleep(5)

for _ in range(523):
  driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  time.sleep(3)

links = driver.find_elements(By.XPATH, "//a[contains(@href, '/in/')]")
urls = set([l.get_attribute('href') for l in links if l.get_attribute('href')])

with open("linkedin_connections_2.txt", "w", encoding="utf-8") as f:
  for url in sorted(urls):
    f.write(url + "\n")

print(f"{len(urls)} links salvos em linkedin_connections_2.txt")

driver.quit()


520 links salvos em linkedin_connections_2.txt


### 📌 Etapa 2: Consolidação das Conexões Extraídas

Após a coleta das conexões de diferentes perfis do LinkedIn (etapa anterior), os dados foram armazenados em arquivos `.txt`. Nesta etapa, realizamos a leitura e unificação dessas informações.

As ações executadas incluem:

1. **Leitura dos Arquivos**  
   Os arquivos `linkedin_connections.txt` e `linkedin_connections_2.txt`, contendo os links extraídos, são abertos e lidos linha por linha.

2. **Limpeza e Unificação**  
   As linhas vazias são removidas e as URLs de ambos os arquivos são combinadas em uma única lista.

3. **Criação do DataFrame**  
   A lista consolidada de URLs é convertida em um `DataFrame` do pandas para facilitar a manipulação e análise dos dados.

4. **Remoção de Duplicatas**  
   URLs repetidas são eliminadas para garantir que cada conexão apareça apenas uma vez.

5. **Salvar os dados em um arquivo `.csv`**  
   Os dados são salvos em um arquivo `.csv` para a segunda parte do processo de coleta.


In [4]:
file_path = "linkedin_connections.txt"
file_path_2 = "linkedin_connections_2.txt"
with open(file_path, "r", encoding="utf-8") as f1, open(file_path_2, "r", encoding="utf-8") as f2:
  urls = [line.strip() for line in f1 if line.strip()] + [line.strip() for line in f2 if line.strip()]

df = pd.DataFrame(urls, columns=["LinkedInURL"]).drop_duplicates().reset_index(drop=True)
df

,LinkedInURL
0,https://www.linkedin.com/in/abdsmaciel/
1,https://www.linkedin.com/in/adan-medeiros-2994...
2,https://www.linkedin.com/in/adelmomaximo/
3,https://www.linkedin.com/in/ademir-guimaraes/
4,https://www.linkedin.com/in/adevancomp/
...,...
649,https://www.linkedin.com/in/ygldc/
650,https://www.linkedin.com/in/yubiferreira/
651,https://www.linkedin.com/in/yuri-barbosa-88453...
652,https://www.linkedin.com/in/yuri-kauan-4674722bb/


In [ ]:
df.to_csv("linkedin_connections.csv", index=False)

### 📌 Etapa 3: Coleta dos Perfis LinkedIn em Formato HTML

Nesta etapa, utilizamos a lista de URLs de conexões (extraídas e salvas anteriormente) para acessar e salvar os perfis completos do LinkedIn em arquivos HTML. Esse processo também foi automatizado com a biblioteca **Selenium**.

As principais ações desta etapa incluem:

1. **Configuração de Login e Leitura dos Dados**
   - Carregamento da lista de URLs a partir do arquivo `linkedin_connections.csv`.
   - Definição das credenciais de acesso (e-mail e senha).
   - Criação da pasta de destino `html_perfis` para armazenar os arquivos HTML.

2. **Abertura do Navegador e Login**
   - O Chrome é iniciado em modo maximizado com Selenium.
   - A automação realiza o login na plataforma do LinkedIn e aguarda até que o feed principal seja carregado.

3. **Navegação pelos Perfis**
   - O script percorre os perfis a partir de um índice inicial customizado (`start_index`), permitindo retomadas em caso de interrupções.
   - Cada perfil é acessado individualmente.

4. **Scroll da Página**
   - Para garantir que todos os elementos do perfil sejam carregados, é realizado um scroll contínuo até o final da página.

5. **Extração e Salvamento**
   - O HTML da página inteira é capturado via `driver.page_source`.
   - O conteúdo é salvo localmente em arquivos nomeados sequencialmente (`profile-<idx>.html`).

6. **Pausa Estratégica**
   - A cada 20 perfis processados, o script pausa por 60 segundos para reduzir o risco de bloqueio automático por parte da plataforma.

Essa etapa resulta em um repositório local completo dos perfis visitados.


In [ ]:
df_raw_linkedin_connections = pd.read_csv('linkedin_connections.csv')
df_raw_linkedin_connections

In [ ]:
EMAIL = "test@gmail.com"
SENHA = "12345"
CSV_PATH = "linkedin_connections.csv" 
OUTPUT_DIR = "html_perfis"

os.makedirs(OUTPUT_DIR, exist_ok=True)

df = pd.read_csv(CSV_PATH)
perfil_urls = df["LinkedInURL"].dropna().unique().tolist()

options = Options()
options.add_argument("--start-maximized")


driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)

try:
    driver.get("https://www.linkedin.com/login")
    wait.until(EC.presence_of_element_located((By.ID, "username"))).send_keys(EMAIL)
    driver.find_element(By.ID, "password").send_keys(SENHA)
    driver.find_element(By.XPATH, "//button[@type='submit']").click()
    wait.until(EC.url_contains("linkedin.com/feed"))
    print("✅ Login realizado com sucesso!")
except Exception as e:
    print(f"❌ Erro no login: {e}")
    driver.quit()
    exit()

start_index = 502

for idx, url in enumerate(perfil_urls[start_index - 1:], start=start_index):
    try:
        print(f"\n🔗 Acessando ({idx}/{len(perfil_urls)}): {url}")
        driver.get(url)
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "h1")))
        time.sleep(2)

        print("⏳ Fazendo scroll...")
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        print("✅ Scroll completo")

        nome_arquivo = f"profile-{idx}.html"
        output_path = os.path.join(OUTPUT_DIR, nome_arquivo)

        html = driver.page_source
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(html)
        print(f"✅ HTML salvo: {output_path}")

        if idx % 20 == 0:
            print("⏸️ Pausando por 60 segundos para evitar bloqueio...")
            time.sleep(60)

    except TimeoutException:
        print(f"⚠️ Timeout ao acessar: {url}")
    except Exception as e:
        print(f"❌ Erro ao processar {url}: {e}")

driver.quit()
print("\n✅ Todos os perfis processados. Navegador fechado.")
